## Introduction

We saw that in the research we are studying, they used Mallows model to generate preferences profiles, which means they didn't use a real-world data for their conclusions, and according to (Boehmer, Faliszewski ,Kraiczy, 2023), the Mallows model may have some issues and we should be careful when using it, especially when generating data for experiments. Therefore, we decided to validate their conclustions in sections 5.2 and 5.3 using a real-world datasets.\
<font color="blue">(Note: In the following sections, we're going to use seq-rule-winner/loser as a shortcut to Sequential-Rule-Winner/Loser.)

## Datasets

We need datasets that have at least 10 candidates and 100 voters, since their graphs are based on profiles with 10 candidates and 100 voters. We picked randomly 41 datasets from the website (https://preflib.simonrey.fr/), and from each one of these datasets we sampled 10 profiles with 10 candidates and 100 voters, and used these profiles to run the simulations, and consequently, generating proper plots.

## Research Conclusions

In section 5.2, the researchers concluded that in seq-plurality-loser and score are similar to each other but seq-plurality-winner is different than the two, and we can see that in their plots, where the line that displays the difference between seq-plurality-loser and score is low, and on the other hand, the line that displays the difference between seq-plurality-winner and each one of seq-plurality-loser and score is high.\
In section 5.3, they compared ranking methods against kemeny, and they concluded that in seq-plurality-winner is the closest to kemeny's ranking for every value of dispersion parameter.

## Our Implementation

In [1]:
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from functools import cmp_to_key
import itertools
from google.colab import drive

drive.mount('/content/drive')

tie_breaking_order = None
tie = None

MessageError: Error: credential propagation was unsuccessful

This function parses the soc file to our preference profile structure.

In [ ]:
def parse_soc_data(soc_file):
    candidates = []
    entries = []
    with open(soc_file, 'r') as file:
        for line in file:
            line_text = line.strip()
            if "ALTERNATIVE NAME" in line_text:
                sushi_name = line_text.split(": ")[-1]
                candidates.append(sushi_name)
            if line_text[0] != "#":
                vote = line_text.split(": ")
                count = int(vote[0])
                order = vote[1].split(',')
                order = [int(x)-1 for x in order]
                for _ in range(count):
                    entries.append(order)
    return entries

This function calculates the normalized dispersion parameter for the given profile.

In [ ]:
def calculate_normalized_dispersion(profile):
    num_voters = len(profile)
    num_candidates = len(profile[0])

    distances = []
    for vote1, vote2 in itertools.combinations(profile, 2):
        distance = sum(1 for c1, c2 in zip(vote1, vote2) if c1 != c2)
        distances.append(distance)

    total_distance = sum(distances)

    max_possible_distance = num_candidates * (num_voters - 1) / 2  # Maximum possible distance
    normalized_dispersion = total_distance / (max_possible_distance * num_voters)

    return normalized_dispersion

In [ ]:
def get_permutation_score(p, pairwise):
    score = 0
    for i in range(0, len(p)-1):
        for j in range(i+1, len(p)):
            score += pairwise[(p[i], p[j])]
    return score

In [ ]:
def get_pairwise_count(profile, a, b):
    count = 0
    for entry in profile:
        count += (entry.index(a) < entry.index(b))
    return count

In [ ]:
def kemeny(profile):
    total = len(profile)
    permutations = list(itertools.permutations(profile[0]))
    pairwise = dict()
    for a in profile[0]:
        for b in profile[0]:
            if a != b:
                if (a, b) not in pairwise.keys():
                    count = get_pairwise_count(profile, a, b)
                    pairwise[(a, b)] = count
                    pairwise[(b, a)] = total - count

    max_profile = 0
    winning_profile = None

    for p in permutations:
        score = get_permutation_score(p, pairwise)
        if score > max_profile:
            max_profile = score
            winning_profile = p
        elif score == max_profile and p[0] < winning_profile[0]:
            winning_profile = p

    return winning_profile

In [ ]:
def plurality(vot, m):
    points = [0] * m
    for v in vot:
        if len(v) > 0:
            points[v[0]] = points[v[0]] + 1
    return points

This function simulates the Sequential-Rule-loser

In [ ]:
def eliminate_bottom(vot, m, rule, tiebreaking):
    tie = 0
    votes = []
    for v in vot:
        vvv = []
        for c in v:
            vvv.append(c)
        votes.append(vvv)

    not_deleted = list(range(m))
    order = [0] * m
    points = rule(vot, m)
    for i in range(m - 1):
        min_relevant = min([points[i] for i in not_deleted])
        cand_to_be_del = [i for i in not_deleted if points[i] == min_relevant]
        if len(cand_to_be_del) > 1:
            tie = tie + 1
        for t in tiebreaking:
            if t in cand_to_be_del:
                delete = t
                break
        order[m - i - 1] = delete
        not_deleted.remove(delete)
        for i in range(len(votes)):
            if delete in votes[i]:
                votes[i].remove(delete)
        points = rule(votes, m)
    order[0] = not_deleted[0]
    return order, tie

This function simulates the Sequential-Rule-Winner

In [ ]:
def eliminate_top(vot, m, rule, tiebreaking):
    tie = 0
    tiebreaking = list(reversed(tiebreaking))
    votes = []
    for v in vot:
        vvv = []
        for c in v:
            vvv.append(c)
        votes.append(vvv)
    not_deleted = list(range(m))
    order = [0] * m
    points = rule(vot, m)
    for i in range(m - 1):
        max_relevant = max([points[i] for i in not_deleted])
        cand_to_be_del = [i for i in not_deleted if points[i] == max_relevant]
        if len(cand_to_be_del) > 1:
            tie = tie + 1
        for t in tiebreaking:
            if t in cand_to_be_del:
                delete = t
                break
        order[i] = delete
        not_deleted.remove(delete)
        for i in range(len(votes)):
            if delete in votes[i]:
                votes[i].remove(delete)
        points = rule(votes, m)
    order[m - 1] = not_deleted[0]
    return order, tie

In [ ]:
def compare(item1, item2):
    if item1[0] > item2[0]:
        return 1
    elif item1[0] < item2[0]:
        return -1
    elif tie_breaking_order.index(item1[1]) < tie_breaking_order.index(item2[1]):
        global tie
        tie = tie + 1
        return 1
    else:
        return -1

In [ ]:
def score_ordering(vot, m, rule, tiebreaking):
    global tie
    tie = 0
    global tie_breaking_order
    tie_breaking_order = tiebreaking
    points = rule(vot, m)
    inversed_points = [-x for x in points]
    to_be_sorted = list(zip(inversed_points, list(range(m))))
    return [x for _, x in sorted(to_be_sorted, key=cmp_to_key(compare))], tie

In [ ]:
def swap_distance(order1, order2, m):
    distance = 0
    for i in range(m):
        for j in range(0, i):
            if not (order1.index(i) < order1.index(j)) == (order2.index(i) < order2.index(j)):
                distance += 1
    return distance / (m * (m - 1) / 2)

This method returns profiles samples given a dataset

In [ ]:
def sample_data(n, m, sample, data_set):
    elections = []

    for _ in range(sample):
        candidates = sorted(random.sample(range(len(data_set[0])), m))
        candidates_set = set(candidates)
        votes = random.sample(range(len(data_set)), n)
        election = []

        for i in votes:
            vote = []
            for v in data_set[i]:
                if v in candidates_set:
                    vote.append(candidates.index(v))

            election.append(vote)

        elections.append(election)

    return elections

This method calculates all the parameters needed for the graph given a profile and rule.

In [ ]:
def run_experiment(election, m, rule):
    tie_breaking = list(np.random.permutation(m))
    orderingSTV, tieSTV = eliminate_bottom(election, m, rule, tie_breaking)
    orderingElimination, tieElimination = eliminate_top(election, m, rule, tie_breaking)
    orderingPoints, tiePoints = score_ordering(election, m, rule, tie_breaking)

    disSTVEli = swap_distance(orderingSTV, orderingElimination, m)
    disSTVPoints = swap_distance(orderingSTV, orderingPoints, m)
    disEliPoints = swap_distance(orderingElimination, orderingPoints, m)

    spearSTVEli, _ = spearmanr(orderingSTV, orderingElimination)
    spearSTVPoints, _ = spearmanr(orderingSTV, orderingPoints)
    spearEliPoints, _ = spearmanr(orderingElimination, orderingPoints)

    central = kemeny(election)

    disSTVnatural = swap_distance(orderingSTV, central, m)
    disPointsnatural = swap_distance(orderingPoints, central, m)
    disElinatural = swap_distance(orderingElimination, central, m)

    spearSTVnatural = spearmanr(orderingSTV, central)[0]
    spearPointsnatural = spearmanr(orderingPoints, central)[0]
    spearElinatural = spearmanr(orderingElimination, central)[0]

    return disSTVEli, disSTVPoints, disEliPoints, spearSTVEli, spearSTVPoints, \
        spearEliPoints, disSTVnatural, disPointsnatural, disElinatural, spearSTVnatural, \
        spearPointsnatural,spearElinatural, tieSTV, tieElimination, tiePoints

This function run the run_expirement function with each profile we sampled and generate the plot.

In [ ]:
def real_experiment(nn, mm, sample, rule, rule_name, data_sets):
    colors = ['darkred', 'darkorange', 'forestgreen', 'dodgerblue', 'darkviolet', "brown"]
    for n in nn:
        for m in mm:
            norm_phis = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
            list_results = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
            samples_phis = []
            for _ in norm_phis:
                samples_phis.append(0)
            for result in list_results:
                for _ in norm_phis:
                    result.append(0)
            result_names = ["Seq.-Lo. vs Seq.-Wi.", "Seq.-Lo. vs Score", "Seq.-Wi. vs Score", "Seq.-Lo. vs Seq.-Wi.",
                  "Seq.-Lo. vs Score", "Seq.-Wi. vs Score", "Kemeny vs Seq.-Lo.", "Kemeny vs Score",
                  "Kemeny vs Seq.-Wi.", "Kemeny vs Seq.-Lo.", "Kemeny vs Score", "Kemeny vs Seq.-Wi.",
                  "Seq.-Lo.", "Seq.-Wi.", "Score"]
            for data_set in data_sets:
                elections = sample_data(n, m, sample, data_set)
                for election in elections:
                    phi = calculate_normalized_dispersion(election)
                    samples_phis[round(phi*10)] += 1
                    results = run_experiment(election, m, rule)
                    for i in range(15):
                        list_results[i][round(phi*10)] += results[i]

            for i in range(15):
                for j in range(len(samples_phis)):
                    if samples_phis[j] > 0:
                        list_results[i][j] /= samples_phis[j]

            for i in range(3):
                plt.plot(norm_phis, list_results[i], label=result_names[i], color=colors[i])
            for i in range(6, 9):
                plt.plot(norm_phis, list_results[i], linestyle='dashed',
                    label=result_names[i], color=colors[i - 3])

    directory = "./plots/"
    if not os.path.exists(directory):
        os.makedirs(directory)
    plt.legend(loc="upper left")
    plt.ylabel('normalized swap distance')
    plt.xlabel('dispersion parameter')
    plt.show()

    plt.close()
    for i in range(12, 15):
        plt.plot(norm_phis, list_results[i],
              label=result_names[i], color=colors[i - 12])
    plt.ylabel('average number of rounds with tie')
    plt.xlabel('dispersion parameter')
    plt.legend(loc="upper left")
    plt.close()

In [ ]:
data = []
for file in os.listdir("./data"):
    data.append(parse_soc_data("data/" + file))

real_experiment([100], [10], 10, plurality, "plurality", data)

## Our Output Plot Explanation

We can see that for the dispersion parameter above 0.9 and below 0.2 the normalized swap distance is 0, and that's because in the real-world data, it's not common for a profile to have nearly 0 or close to 1 dispersion parameter, at least for the data we found. Hence, we can see that our plots is more random because we based our plots on a smaller number of profiles, and usually in real-world observations are more chaotic.

## Research's Plot

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
image_path = "./research conclusions/plurality_100_10_mallows.png"
image = Image.open(image_path)
plt.imshow(image)
plt.axis('off')
plt.show()

## Discussion

According to the plots we generated, we can see that in general we can have the same conclusions:
- For section 5.2: Seq-plurality-loser is more similar to the score (The line of "seq-plurality-loser vs score" is low in our plot), which indicates a high similarity between the two methods, and on the other hand, we can see that the seq-plurality-winner is different than the other to methods since the two lines that demonstrate the differenece between seq-plurality-winner and the other two methods are much higher.
- For section 5.3: In our plot, we can see that seq-plurality-winner is the closest to kemeny method since the line that exhibits their distance is lower than the lines that exhibits the distance between kemeny method and each one of seq-plurality-loser and score methods.

Therefore, the results that were shown in the research section 5 can be extended to real-world data.